In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# !unzip "/content/drive/MyDrive/open.zip"

In [2]:
# 데이콘 리더 보드 점수 => 	0.4800326021
# b6에 맞는 이미지 사이즈 조정 필요 => 528

## Import

In [3]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

# pip3 install torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

# pip3 install albumentations --user
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

# pip3 install torchvision
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [5]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

## Fixed RandomSeed

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [7]:
# 학습용 이미지 폴더 경로 수정
# all_img_list = glob.glob(os.path.join(os.path.pardir, os.path.pardir, 'open', 'train', ) + '/*/*')
all_img_list = glob.glob('/content/train/*/*')
print('all_img_list :', all_img_list)

all_img_list : ['/content/train/석고수정/13.png', '/content/train/석고수정/11.png', '/content/train/석고수정/6.png', '/content/train/석고수정/20.png', '/content/train/석고수정/25.png', '/content/train/석고수정/29.png', '/content/train/석고수정/42.png', '/content/train/석고수정/41.png', '/content/train/석고수정/27.png', '/content/train/석고수정/33.png', '/content/train/석고수정/14.png', '/content/train/석고수정/48.png', '/content/train/석고수정/3.png', '/content/train/석고수정/40.png', '/content/train/석고수정/22.png', '/content/train/석고수정/53.png', '/content/train/석고수정/30.png', '/content/train/석고수정/10.png', '/content/train/석고수정/50.png', '/content/train/석고수정/39.png', '/content/train/석고수정/2.png', '/content/train/석고수정/34.png', '/content/train/석고수정/7.png', '/content/train/석고수정/28.png', '/content/train/석고수정/47.png', '/content/train/석고수정/0.png', '/content/train/석고수정/21.png', '/content/train/석고수정/23.png', '/content/train/석고수정/5.png', '/content/train/석고수정/55.png', '/content/train/석고수정/56.png', '/content/train/석고수정/24.png', '/content/train/석고수정/43.png', 

In [8]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list

# 학습용 이미지 폴더 경로 수정
# df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[4])
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[3])
print('df :', df)

df :                         img_path label
0     /content/train/석고수정/13.png  석고수정
1     /content/train/석고수정/11.png  석고수정
2      /content/train/석고수정/6.png  석고수정
3     /content/train/석고수정/20.png  석고수정
4     /content/train/석고수정/25.png  석고수정
...                          ...   ...
3452   /content/train/면불량/97.png   면불량
3453   /content/train/면불량/17.png   면불량
3454    /content/train/면불량/9.png   면불량
3455   /content/train/면불량/49.png   면불량
3456   /content/train/면불량/98.png   면불량

[3457 rows x 2 columns]


In [9]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])
print('train :', train)

train :                         img_path label
1352   /content/train/훼손/609.png    훼손
119    /content/train/훼손/565.png    훼손
351    /content/train/훼손/907.png    훼손
629    /content/train/훼손/218.png    훼손
335    /content/train/훼손/309.png    훼손
...                          ...   ...
975   /content/train/훼손/1388.png    훼손
3045   /content/train/오염/419.png    오염
3439   /content/train/면불량/15.png   면불량
797    /content/train/훼손/185.png    훼손
1551  /content/train/몰딩수정/12.png  몰딩수정

[2419 rows x 2 columns]


## Label-Encoding

In [10]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])
print('train :', train)

train :                         img_path  label
1352   /content/train/훼손/609.png     18
119    /content/train/훼손/565.png     18
351    /content/train/훼손/907.png     18
629    /content/train/훼손/218.png     18
335    /content/train/훼손/309.png     18
...                          ...    ...
975   /content/train/훼손/1388.png     18
3045   /content/train/오염/419.png     10
3439   /content/train/면불량/15.png      6
797    /content/train/훼손/185.png     18
1551  /content/train/몰딩수정/12.png      7

[2419 rows x 2 columns]


## CustomDataset

In [11]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        # TypeError: image must be numpy array type
        # issue: None => 경로에 한글명 존재
        # image = cv2.imread(img_path)
        # print('image :', image)

        # solved
        img_array = np.fromfile(img_path, np.uint8)
        image = cv2.imdecode(img_array, cv2.IMREAD_UNCHANGED)
        # print('image :', image)

        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [12]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [13]:
print('train[img_path].values :', train['img_path'].values)
print('train[label].values :', train['label'].values)
# print('train_transform :', train_transform)

train_dataset = CustomDataset(
    train['img_path'].values, train['label'].values, train_transform)

train_loader = DataLoader(
    train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values,
                            val['label'].values, test_transform)

val_loader = DataLoader(
    val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

train[img_path].values : ['/content/train/훼손/609.png' '/content/train/훼손/565.png'
 '/content/train/훼손/907.png' ... '/content/train/면불량/15.png'
 '/content/train/훼손/185.png' '/content/train/몰딩수정/12.png']
train[label].values : [18 18 18 ...  6 18  7]


## Model Define

In [14]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b6(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

## Train

In [15]:
def validation(model, criterion, val_loader, device):
    
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):

            imgs = imgs.float().to(device)
            
            # RuntimeError: expected scalar type Long but found Int
            # labels = labels.to(device)

            # solved
            labels = labels.type(torch.LongTensor).to(device) 
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

In [16]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):

    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):

        model.train()
        train_loss = []

        for imgs, labels in tqdm(iter(train_loader)):

            imgs = imgs.float().to(device)

            # RuntimeError: expected scalar type Long but found Int
            # labels = labels.to(device)

            # solved
            labels = labels.type(torch.LongTensor).to(device) 
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

## Run!!

In [17]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

Downloading: "https://download.pytorch.org/models/efficientnet_b6_lukemelas-c76e70fd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b6_lukemelas-c76e70fd.pth
100%|██████████| 165M/165M [00:10<00:00, 15.8MB/s]


  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [1], Train Loss : [1.45277] Val Loss : [1.01421] Val Weighted F1 Score : [0.67159]


  0%|          | 0/76 [00:00<?, ?it/s]

...

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [9], Train Loss : [0.00707] Val Loss : [1.09225] Val Weighted F1 Score : [0.79452]


  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [10], Train Loss : [0.00697] Val Loss : [1.05876] Val Weighted F1 Score : [0.79713]


## Inference

In [18]:
# 테스트 csv 파일 경로 수정
# test = pd.read_csv('../../open/test.csv')
test = pd.read_csv('/content/test.csv')

In [19]:
# ======== 테스트 이미지 파일 경로 수정 ========

# test_img_paths = []

# for i in test['img_path'].values:

#     test_img_path = '../../open/' + i.replace('./', '')

#     test_img_paths.append(test_img_path)

# print('test_img_paths :', test_img_paths)

# test_dataset = CustomDataset(test_img_paths, None, test_transform)
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)

# =============================================

test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [20]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [21]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

## Submission

In [25]:
# 제출용 추론 결과 csv 파일 경로 수정
# submit = pd.read_csv('../../open/sample_submission.csv')
submit = pd.read_csv('/content/sample_submission.csv')

In [26]:
submit['label'] = preds

In [27]:
submit.to_csv('./baseline_submit.csv', index=False)